In [ ]:
import pandas as pd
from pathlib import Path
import re
from collections import defaultdict
from tqdm import tqdm


In [ ]:
# Define paths
gtex_path = Path("datasets/raw/GTEx")
pharmgkb_path = Path("datasets/raw/PharmKGB") 
fda_path = Path("datasets/raw/FDA Orange Book")
faers_path = Path("datasets/raw/FAERS")

# GTEx
gtex_liver = pd.read_csv(gtex_path / "gene_tpm_v10_liver.gct/gene_tpm_2022-06-06_v10_liver.gct", sep="\t", skiprows=2, low_memory=False)
gtex_kidney = pd.read_csv(gtex_path / "gene_tpm_v10_kidney_cortex.gct/gene_tpm_2022-06-06_v10_kidney_cortex.gct", sep="\t", skiprows=2, low_memory=False)
gtex_colon = pd.read_csv(gtex_path / "gene_tpm_v10_colon_transverse.gct/gene_tpm_2022-06-06_v10_colon_transverse.gct", sep="\t", skiprows=2, low_memory=False)
gtex_intestine = pd.read_csv(gtex_path / "gene_tpm_v10_small_intestine_terminal_ileum.gct/gene_tpm_2022-06-06_v10_small_intestine_terminal_ileum.gct", sep="\t", skiprows=2, low_memory=False)
gtex_blood = pd.read_csv(gtex_path / "gene_tpm_v10_whole_blood.gct/gene_tpm_2022-06-06_v10_whole_blood.gct", sep="\t", skiprows=2, low_memory=False)

# PharmGKB
pharmgkb_drugs = pd.read_csv(pharmgkb_path / "drugs/drugs.tsv", sep="\t", low_memory=False)
pharmgkb_genes = pd.read_csv(pharmgkb_path / "genes/genes.tsv", sep="\t", low_memory=False)
pharmgkb_relationships = pd.read_csv(pharmgkb_path / "relationships/relationships.tsv", sep="\t", low_memory=False)

# FDA
fda_products = pd.read_csv(fda_path / "FDA_PRODUCTS.csv", low_memory=False)
fda_applications = pd.read_csv(fda_path / "FDA_APPLICATIONS.csv", low_memory=False)

# FAERS
faers_demographics = pd.read_csv(faers_path / "FAERS_DEMOGRAPHICS.csv", low_memory=False)
faers_drugs = pd.read_csv(faers_path / "FAERS_DRUGS.csv", low_memory=False)
faers_reactions = pd.read_csv(faers_path / "FAERS_REACTIONS.csv", low_memory=False)


In [ ]:
# Shapes
print("GTEx liver:", gtex_liver.shape)
print("GTEx kidney:", gtex_kidney.shape)
print("GTEx colon:", gtex_colon.shape)
print("GTEx intestine:", gtex_intestine.shape)
print("GTEx blood:", gtex_blood.shape)

print("PharmGKB drugs:", pharmgkb_drugs.shape)
print("PharmGKB genes:", pharmgkb_genes.shape)
print("PharmGKB relationships:", pharmgkb_relationships.shape)

print("FDA products:", fda_products.shape)
print("FDA applications:", fda_applications.shape)

print("FAERS demographics:", faers_demographics.shape)
print("FAERS drugs:", faers_drugs.shape)
print("FAERS reactions:", faers_reactions.shape)

# Samples
print(gtex_liver.head(3))
print(pharmgkb_drugs.head(3))
print(fda_products.head(3))
print(faers_drugs.head(3))


In [ ]:
# GTEx (only show Name + Description since rest are expression values)
print("GTEx liver:")
print("  Name - unique:", gtex_liver['Name'].nunique(), " null:", gtex_liver['Name'].isna().sum())
print("  Description - unique:", gtex_liver['Description'].nunique(), " null:", gtex_liver['Description'].isna().sum())

# PharmGKB
print("\nPharmGKB drugs:")
for col in ['Name', 'Generic Names', 'Trade Names', 'ATC Identifiers', 'PubChem Compound Identifiers']:
    print(f"  {col} - unique:", pharmgkb_drugs[col].nunique(), " null:", pharmgkb_drugs[col].isna().sum())

print("\nPharmGKB genes:")
for col in ['Ensembl Id', 'Symbol', 'Name']:
    print(f"  {col} - unique:", pharmgkb_genes[col].nunique(), " null:", pharmgkb_genes[col].isna().sum())

print("\nPharmGKB relationships:")
for col in ['Entity1_name', 'Entity1_type', 'Entity2_name', 'Entity2_type']:
    print(f"  {col} - unique:", pharmgkb_relationships[col].nunique(), " null:", pharmgkb_relationships[col].isna().sum())

# FDA
print("\nFDA products:")
for col in ['drug_name', 'active_ingredient']:
    print(f"  {col} - unique:", fda_products[col].nunique(), " null:", fda_products[col].isna().sum())

print("\nFDA applications:")
for col in ['appl_no', 'sponsor_name']:
    print(f"  {col} - unique:", fda_applications[col].nunique(), " null:", fda_applications[col].isna().sum())

# FAERS
print("\nFAERS drugs:")
for col in ['drugname', 'prod_ai']:
    print(f"  {col} - unique:", faers_drugs[col].nunique(), " null:", faers_drugs[col].isna().sum())

print("\nFAERS reactions:")
print("  pt - unique:", faers_reactions['pt'].nunique(), " null:", faers_reactions['pt'].isna().sum())

print("\nFAERS demographics:")
for col in ['age', 'sex', 'reporter_country']:
    print(f"  {col} - unique:", faers_demographics[col].nunique(), " null:", faers_demographics[col].isna().sum())


In [ ]:
# Genes: strip version from GTEx Ensembl IDs
for df in [gtex_liver, gtex_kidney, gtex_colon, gtex_intestine, gtex_blood]:
    df['Name'] = df['Name'].astype(str).str.split('.').str[0]

pharmgkb_genes['Ensembl Id'] = pharmgkb_genes['Ensembl Id'].astype(str).str.split('.').str[0]

# Drug Normalization Function
def normalize_drug_name(name):
    if pd.isna(name):
        return None
    name = str(name).lower().strip()
    name = re.sub(r'[^a-z0-9\s]', ' ', name)  # keep alphanumeric
    name = re.sub(r'\b(mg|mcg|g|ml|tab|tablet|capsule|solution|injection|suspension|cream|ointment|drops|%|\d+)\b', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip()

# Apply to PharmGKB
pharmgkb_drugs['norm_name'] = pharmgkb_drugs['Name'].apply(normalize_drug_name)

# Apply to FDA
fda_products['norm_drug_name'] = fda_products['drug_name'].apply(normalize_drug_name)
fda_products['norm_active_ingredient'] = fda_products['active_ingredient'].apply(normalize_drug_name)

# Apply to FAERS
faers_drugs['norm_drugname'] = faers_drugs['drugname'].apply(normalize_drug_name)
faers_drugs['norm_prod_ai'] = faers_drugs['prod_ai'].apply(normalize_drug_name)

# Clean FAERS adverse events (pt)
faers_reactions['pt'] = faers_reactions['pt'].astype(str).str.strip().str.lower()


In [ ]:
# Unique normalized drug sets
pharmgkb_drug_set = set(pharmgkb_drugs['norm_name'].dropna())
fda_drug_set = set(fda_products['norm_drug_name'].dropna()).union(
    set(fda_products['norm_active_ingredient'].dropna())
)
faers_drug_set = set(faers_drugs['norm_drugname'].dropna()).union(
    set(faers_drugs['norm_prod_ai'].dropna())
)

# Overlaps
pharmgkb_fda_overlap = pharmgkb_drug_set & fda_drug_set
pharmgkb_faers_overlap = pharmgkb_drug_set & faers_drug_set
fda_faers_overlap = fda_drug_set & faers_drug_set
three_way_overlap = pharmgkb_drug_set & fda_drug_set & faers_drug_set

print("PharmGKB drugs:", len(pharmgkb_drug_set))
print("FDA drugs:", len(fda_drug_set))
print("FAERS drugs:", len(faers_drug_set))

print("PharmGKB ∩ FDA:", len(pharmgkb_fda_overlap))
print("PharmGKB ∩ FAERS:", len(pharmgkb_faers_overlap))
print("FDA ∩ FAERS:", len(fda_faers_overlap))
print("Three-way overlap:", len(three_way_overlap))

# Genes: overlap between GTEx and PharmGKB
gtex_gene_set = set(gtex_liver['Description'].dropna().astype(str))
pharmgkb_gene_set = set(pharmgkb_genes['Symbol'].dropna().astype(str))
gene_overlap = gtex_gene_set & pharmgkb_gene_set

print("GTEx genes:", len(gtex_gene_set))
print("PharmGKB genes:", len(pharmgkb_gene_set))
print("Gene overlap:", len(gene_overlap))


In [ ]:
# Reduce PharmGKB drugs
pharmgkb_drugs_reduced = pharmgkb_drugs[
    pharmgkb_drugs['norm_name'].isin(pharmgkb_fda_overlap | pharmgkb_faers_overlap)
].copy()

# Reduce PharmGKB genes
pharmgkb_genes_reduced = pharmgkb_genes[
    pharmgkb_genes['Symbol'].isin(gene_overlap) | pharmgkb_genes['Ensembl Id'].isin(gtex_liver['Name'])
].copy()

# Reduce PharmGKB relationships
pharmgkb_relationships_reduced = pharmgkb_relationships[
    (pharmgkb_relationships['Entity1_name'].isin(pharmgkb_drugs['Name'])) |
    (pharmgkb_relationships['Entity2_name'].isin(pharmgkb_genes['Symbol']))
].copy()

# Reduce FDA
fda_products_reduced = fda_products[
    (fda_products['norm_drug_name'].isin(pharmgkb_fda_overlap)) |
    (fda_products['norm_active_ingredient'].isin(pharmgkb_fda_overlap))
].copy()

# Reduce FAERS drugs
faers_drugs_reduced = faers_drugs[
    (faers_drugs['norm_drugname'].isin(pharmgkb_faers_overlap)) |
    (faers_drugs['norm_prod_ai'].isin(pharmgkb_faers_overlap))
].copy()

# Reduce FAERS reactions
faers_reactions_reduced = faers_reactions[
    faers_reactions['primaryid'].isin(faers_drugs_reduced['primaryid'])
].copy()

# Reduce FAERS demographics
faers_demographics_reduced = faers_demographics[
    faers_demographics['primaryid'].isin(faers_drugs_reduced['primaryid'])
].copy()

# Reduce GTEx
gtex_liver_reduced = gtex_liver[gtex_liver['Description'].isin(gene_overlap)].copy()
gtex_kidney_reduced = gtex_kidney[gtex_kidney['Description'].isin(gene_overlap)].copy()
gtex_colon_reduced = gtex_colon[gtex_colon['Description'].isin(gene_overlap)].copy()
gtex_intestine_reduced = gtex_intestine[gtex_intestine['Description'].isin(gene_overlap)].copy()
gtex_blood_reduced = gtex_blood[gtex_blood['Description'].isin(gene_overlap)].copy()

# Check reduced dataset sizes
print("PharmGKB drugs reduced:", pharmgkb_drugs_reduced.shape)
print("PharmGKB genes reduced:", pharmgkb_genes_reduced.shape)
print("PharmGKB relationships reduced:", pharmgkb_relationships_reduced.shape)
print("FDA products reduced:", fda_products_reduced.shape)
print("FAERS drugs reduced:", faers_drugs_reduced.shape)
print("FAERS reactions reduced:", faers_reactions_reduced.shape)
print("FAERS demographics reduced:", faers_demographics_reduced.shape)
print("GTEx liver reduced:", gtex_liver_reduced.shape)
print("GTEx kidney reduced:", gtex_kidney_reduced.shape)
print("GTEx colon reduced:", gtex_colon_reduced.shape)
print("GTEx intestine reduced:", gtex_intestine_reduced.shape)
print("GTEx blood reduced:", gtex_blood_reduced.shape)


In [ ]:
datasets_reduced = {
    "PharmGKB drugs": pharmgkb_drugs_reduced,
    "PharmGKB genes": pharmgkb_genes_reduced,
    "PharmGKB relationships": pharmgkb_relationships_reduced,
    "FDA products": fda_products_reduced,
    "FAERS drugs": faers_drugs_reduced,
    "FAERS reactions": faers_reactions_reduced,
    "FAERS demographics": faers_demographics_reduced,
    "GTEx liver": gtex_liver_reduced,
    "GTEx kidney": gtex_kidney_reduced,
    "GTEx colon": gtex_colon_reduced,
    "GTEx intestine": gtex_intestine_reduced,
    "GTEx blood": gtex_blood_reduced
}

for name, df in datasets_reduced.items():
    print(f"\n{name}")
    print(df.isna().sum())


In [ ]:
print("FAERS drugs reduced columns:", list(faers_drugs_reduced.columns))
print("FAERS reactions reduced columns:", list(faers_reactions_reduced.columns))
print("FAERS demographics reduced columns:", list(faers_demographics_reduced.columns))

print("\nPharmGKB relationships reduced columns:", list(pharmgkb_relationships_reduced.columns))

print("\nFDA products reduced columns:", list(fda_products_reduced.columns))
print("FDA applications reduced columns:", list(fda_applications.columns))

print("\nGTEx liver reduced columns (first 5):", list(gtex_liver_reduced.columns)[:5], "... total:", len(gtex_liver_reduced.columns))


In [ ]:
# Testing one drug–gene pair (simvastatin ↔ CYP3A4)

target_drug = "simvastatin"
target_gene = "CYP3A4"

# GTEx expression extraction
def get_expression(gene_symbol):
    expr = {}
    for tissue_name, df in {
        "liver": gtex_liver_reduced,
        "kidney": gtex_kidney_reduced,
        "colon": gtex_colon_reduced,
        "intestine": gtex_intestine_reduced,
        "blood": gtex_blood_reduced
    }.items():
        row = df[df["Description"] == gene_symbol]
        if not row.empty:
            # take mean across all samples for this gene
            expr[f"expr_{tissue_name}"] = row.iloc[:, 2:].mean(axis=1).values[0]
        else:
            expr[f"expr_{tissue_name}"] = np.nan
    return expr

expr_values = get_expression(target_gene)

# FAERS cases for the drug
faers_cases = faers_drugs_reduced[faers_drugs_reduced["norm_drugname"] == target_drug]

# Join with reactions
faers_cases = faers_cases.merge(
    faers_reactions_reduced, on=["primaryid", "caseid"], how="left"
)

# Join with demographics
faers_cases = faers_cases.merge(
    faers_demographics_reduced[["primaryid", "caseid", "age", "sex", "event_dt"]],
    on=["primaryid", "caseid"], how="left"
)

# PharmGKB relationship metadata
rel = pharmgkb_relationships_reduced[
    ((pharmgkb_relationships_reduced["Entity1_name"] == target_drug) & 
     (pharmgkb_relationships_reduced["Entity2_name"] == target_gene)) |
    ((pharmgkb_relationships_reduced["Entity1_name"] == target_gene) & 
     (pharmgkb_relationships_reduced["Entity2_name"] == target_drug))
]

if not rel.empty:
    evidence_level = rel["Evidence"].iloc[0]
    association_type = rel["Association"].iloc[0]
else:
    evidence_level, association_type = None, None

# 4. FDA application metadata
fda_meta = fda_products_reduced[fda_products_reduced["norm_drug_name"] == target_drug]
if not fda_meta.empty:
    appl_no = fda_meta["appl_no"].iloc[0]
    approval_year = None  # placeholder until appl year is parsed
    regulatory_complexity = fda_meta.groupby("appl_no").size().mean()
else:
    approval_year, regulatory_complexity = None, None

# Assemble final dataset (subset for now)
prototype_rows = []
for _, row in faers_cases.head(10).iterrows():
    prototype_rows.append({
        "drug": target_drug,
        "gene": target_gene,
        "case_id": row["caseid"],
        **expr_values,
        "evidence_level": evidence_level,
        "association_type": association_type,
        "age": row["age"],
        "sex": row["sex"],
        "adverse_event": row["pt"],
        "severity": None,  # to be derived from pt later
        "approval_year": approval_year,
        "regulatory_complexity": regulatory_complexity,
        "outcome_severity": None,  # placeholder
        "time_to_event": None       # placeholder
    })

prototype_df = pd.DataFrame(prototype_rows)
print(prototype_df.head(10))


In [ ]:
fda_submissions = pd.read_csv(fda_path / "FDA_SUBMISSIONS.csv", low_memory=False)
fda_application_docs = pd.read_csv(fda_path / "FDA_APPLICATION_DOCS.csv", low_memory=False)

date_cols = ["event_dt", "mfr_dt", "init_fda_dt", "fda_dt", "rept_dt"]
for col in date_cols:
    faers_demographics_reduced[col] = pd.to_datetime(
        faers_demographics_reduced[col], errors="coerce"
    )


In [ ]:
# Trying direct integration

# Severity Mapping Dictionary
severity_map = {
    "death": "life-threatening",
    "shock": "life-threatening",
    "sepsis": "severe",
    "anaphylaxis": "severe",
    "cancer": "severe",
    "stroke": "severe",
    "infarction": "severe",
    "fracture": "moderate",
    "vomiting": "moderate",
    "headache": "mild",
    "nausea": "mild",
    "fatigue": "mild"
}
severity_rank = {"mild": 1, "moderate": 2, "severe": 3, "life-threatening": 4}

def map_severity(pt: str):
    if not isinstance(pt, str):
        return None
    for keyword, label in severity_map.items():
        if keyword in pt.lower():
            return label
    return "unknown"

# Event Categorization

# Expanded FAERS event categories
event_categories_map = {
    "admin_use_issues": [
        "off label use",
        "product use issue",
        "product use in unapproved indication",
        "drug ineffective",
        "drug ineffective for unapproved indication",
        "treatment failure",
        "therapeutic response decreased",
        "therapeutic response shortened",
        "therapeutic product effect decreased",
        "therapeutic product effect incomplete",
        "therapy interrupted",
        "therapy non-responder",
        "treatment noncompliance",
        "intentional product misuse",
        "intentional product use issue",
        "intentional dose omission",
        "product dose omission issue",
        "product dose omission in error",
        "drug dose omission by device",
        "incorrect dose administered",
        "extra dose administered",
        "overdose",
        "wrong technique in product usage process",
        "product storage error",
        "device issue",
        "device difficult to use",
        "injection site pain",
        "injection site erythema",
        "injection site swelling",
        "injection site haemorrhage",
        "injection site hemorrhage",
        "injection site bruising",
        "accidental exposure to product",
        "exposure via skin contact",
        "inappropriate schedule of product administration",
        "product use in unapproved",
        "no adverse event",
        "adverse drug reaction",
        "adverse event",
        "hospitalisation",
        "hospitalization",
        "surgery",
        "patient elopement"
    ],

    "general_symptoms": [
        "fatigue",
        "malaise",
        "pyrexia",
        "fever",
        "chills",
        "pain",
        "pain in extremity",
        "back pain",
        "chest discomfort",
        "discomfort",
        "feeling abnormal",
        "asthenia",
        "gait disturbance",
        "mobility decreased",
        "fall",
        "insomnia",
        "sleep disorder",
        "somnolence",
        "dizziness",
        "tremor",
        "stress"
    ],

    "respiratory": [
        "dyspnoea",
        "dyspnea",
        "wheezing",
        "asthma",
        "cough",
        "productive cough",
        "bronchitis",
        "pneumonia",
        "upper respiratory tract infection",
        "lower respiratory tract infection",
        "nasopharyngitis",
        "rhinorrhoea",
        "rhinorrhea",
        "nasal congestion",
        "dysphonia",
        "lung disorder",
        "oxygen saturation decreased",
        "pulmonary embolism",
        "wheez"
    ],

    "cardiac": [
        "cardiac failure",
        "cardiac disorder",
        "myocardial infarction",
        "atrial fibrillation",
        "arrhythmia",
        "tachycardia",
        "palpitations",
        "heart rate increased",
        "chest pain",
        "cerebrovascular accident",
        "left ventricular hypertrophy"
    ],

    "vascular_hematologic": [
        "thrombosis",
        "haemorrhage",
        "hemorrhage",
        "epistaxis",
        "platelet count decreased",
        "thrombocytopenia",
        "anaemia",
        "anemia",
        "haemoglobin decreased",
        "hemoglobin decreased",
        "white blood cell count decreased",
        "neutropenia",
        "myelosuppression",
        "febrile neutropenia"
    ],

    "neuro": [
        "headache",
        "migraine",
        "seizure",
        "loss of consciousness",
        "syncope",
        "tremor",
        "hypoaesthesia",
        "hypoesthesia",
        "paraesthesia",
        "paresthesia",
        "memory impairment",
        "confusional state",
        "visual impairment",
        "vision blurred",
        "gait inability"
    ],

    "psychiatric": [
        "anxiety",
        "depression",
        "suicidal ideation",
        "confusional state",
        "insomnia",
        "sleep disorder",
        "stress",
        "delusion"
    ],

    "gi": [
        "nausea",
        "vomit",
        "diarrhoea",
        "diarrhea",
        "constipation",
        "abdominal pain",
        "abdominal pain upper",
        "abdominal discomfort",
        "abdominal distension",
        "dyspepsia",
        "dysphagia",
        "stomatitis",
        "gastrooesophageal reflux disease",
        "gastroesophageal reflux disease",
        "crohn's disease",
        "colitis ulcerative",
        "haematochezia",
        "hematochezia",
        "mucosal ulceration",
        "gastrointestinal disorder",
        "gastrointestinal surgery"
    ],

    "hepatic": [
        "hepatic enzyme increased",
        "liver",
        "hepatic",
            "bilirubin"
    ],

    "renal": [
        "acute kidney injury",
        "renal failure",
        "renal impairment",
        "renal",
        "kidney",
        "nephro"
    ],

    "dermatologic_immunologic": [
        "pruritus",
        "rash",
        "erythema",
        "dermatitis atopic",
        "eczema",
        "psoriasis",
        "skin exfoliation",
        "urticaria",
        "alopecia",
        "blister",
        "dry skin",
        "dry mouth",
        "hypersensitivity",
        "drug hypersensitivity",
        "infusion related reaction",
        "peripheral swelling",
        "oedema peripheral",
        "edema peripheral",
        "oedema",
        "edema",
        "periorbital oed",
        "periorbital ed"
    ],

    "musculoskeletal": [
        "arthralgia",
        "myalgia",
        "joint swelling",
        "musculoskeletal stiffness",
        "muscle spasms",
        "muscular weakness",
        "arthropathy",
        "rheumatoid arthritis",
        "psoriatic arthropathy",
        "back pain",
        "pain in extremity"
    ],

    "ophthalmologic": [
        "dry eye",
        "visual impairment",
        "vision blurred",
        "cataract",
        "retinal oed",
        "retinal ed",
        "eye contusion"
    ],

    "ent_oral": [
        "oropharyngeal pain",
        "dry mouth",
        "dysphagia",
        "stomatitis",
        "nasal congestion",
        "rhinorrhoea",
        "rhinorrhea",
        "dysphonia"
    ],

    "endocrine_metabolic": [
        "blood glucose increased",
        "weight decreased",
        "weight increased",
        "hyperhidrosis",
        "hyperandrogen"
    ],

    "infection_inflammation": [
        "infection",
        "influenza",
        "influenza like illness",
        "covid-19",
        "sepsis",
        "urinary tract infection",
        "sinusitis",
        "inflammation",
        "pneumonia",
        "bronchitis"
    ],

    "oncology_disease_course": [
        "malignant neoplasm progression",
        "disease progression",
        "disease recurrence"
    ],

    "pregnancy_perinatal": [
        "maternal exposure during pregnancy",
        "exposure during pregnancy"
    ],

    "vitals_labs_bp": [
        "blood pressure increased",
        "hypertension",
        "hypotension",
        "blood cholesterol increased",
        "oxygen saturation decreased"
    ]
}

def categorize_event(pt: str):
    if not isinstance(pt, str):
        return ["other"]
    cats = []
    for cat, kws in event_categories_map.items():
        if any(kw in pt.lower() for kw in kws):
            cats.append(cat)
    return cats if cats else ["other"]

# FDA Approval Year Extractor
def get_approval_year(drug):
    try:
        meta = fda_products_reduced[fda_products_reduced["norm_drug_name"] == drug]
        if meta.empty:
            return None
        appl_nos = meta["appl_no"].dropna().unique()

        # 1. FDA_SUBMISSIONS
        subs = fda_submissions[fda_submissions["appl_no"].isin(appl_nos)]
        subs = subs[
            (subs["submission_type"] == "ORIG") &
            (subs["submission_status"] == "AP")
        ]
        if not subs.empty:
            dates = pd.to_datetime(subs["submission_status_date"], errors="coerce").dropna()
            if not dates.empty:
                return int(dates.min().year)

        # 2. FDA_APPLICATION_DOCS
        docs = fda_application_docs[fda_application_docs["appl_no"].isin(appl_nos)]
        if not docs.empty:
            dates = pd.to_datetime(docs["application_docs_date"], errors="coerce").dropna()
            if not dates.empty:
                return int(dates.min().year)

        # 3. fallback: FAERS earliest event year
        faers_dates = faers_demographics_reduced.loc[
            faers_demographics_reduced["caseid"].isin(
                faers_drugs_reduced[faers_drugs_reduced["norm_drugname"] == drug]["caseid"]
            ),
            "event_dt"
        ]
        if not faers_dates.dropna().empty:
            return int(str(faers_dates.min())[:4])
    except Exception:
        return None
    return None

# Expression Extractor
def get_expression(gene_symbol):
    expr = {}
    tissue_dfs = {
        "liver": gtex_liver_reduced,
        "kidney": gtex_kidney_reduced,
        "colon": gtex_colon_reduced,
        "intestine": gtex_intestine_reduced,
        "blood": gtex_blood_reduced
    }
    for tissue_name, df in tissue_dfs.items():
        row = df[df["Description"] == gene_symbol]
        expr[f"expr_{tissue_name}"] = row.iloc[:, 2:].mean(axis=1).values[0] if not row.empty else np.nan
    return expr

# Build Drug-Gene Pairs
pharmgkb_drug_names = set(pharmgkb_drugs_reduced["norm_name"].dropna().unique())
fda_drug_names = set(fda_products_reduced["norm_drug_name"].dropna().unique())
faers_drug_names = set(faers_drugs_reduced["norm_drugname"].dropna().unique())
gtex_gene_names = set(gtex_liver_reduced["Description"].dropna().unique())
pharmgkb_gene_names = set(pharmgkb_genes_reduced["Symbol"].dropna().unique())

three_way_drugs = pharmgkb_drug_names & fda_drug_names & faers_drug_names
gene_overlap = gtex_gene_names & pharmgkb_gene_names

print(f"PharmGKB drugs: {len(pharmgkb_drug_names)}")
print(f"FDA drugs: {len(fda_drug_names)}")
print(f"FAERS drugs: {len(faers_drug_names)}")
print(f"Three-way drug overlap: {len(three_way_drugs)}")
print(f"Gene overlap: {len(gene_overlap)}")

drug_gene_pairs = set()
for _, row in pharmgkb_relationships_reduced.iterrows():
    d, g = None, None
    if row["Entity1_type"] == "Chemical" and row["Entity2_type"] == "Gene":
        d, g = row["Entity1_name"], row["Entity2_name"]
    elif row["Entity1_type"] == "Gene" and row["Entity2_type"] == "Chemical":
        d, g = row["Entity2_name"], row["Entity1_name"]
    if d and g:
        norm_d = normalize_drug_name(d)
        if norm_d in three_way_drugs and g in gene_overlap:
            drug_gene_pairs.add((norm_d, g))

print(f"Available drug-gene pairs: {len(drug_gene_pairs)}")

# Build Aggregated Dataset
sample_pairs = list(drug_gene_pairs)[:10]  # subset for testing
agg_rows = []

for drug, gene in tqdm(sample_pairs, desc="Building aggregated dataset"):
    expr_values = get_expression(gene)
    approval_year = get_approval_year(drug)

    faers_cases = faers_drugs_reduced[faers_drugs_reduced["norm_drugname"] == drug]
    if faers_cases.empty:
        continue

    faers_cases = faers_cases.merge(faers_reactions_reduced, on=["primaryid", "caseid"], how="inner")
    faers_cases = faers_cases.merge(
        faers_demographics_reduced[["primaryid","caseid","age","sex"]],
        on=["primaryid","caseid"], how="left"
    )

    rel = pharmgkb_relationships_reduced[
        ((pharmgkb_relationships_reduced["Entity1_name"] == drug) &
         (pharmgkb_relationships_reduced["Entity2_name"] == gene)) |
        ((pharmgkb_relationships_reduced["Entity1_name"] == gene) &
         (pharmgkb_relationships_reduced["Entity2_name"] == drug))
    ]
    evidence_level = rel["Evidence"].iloc[0] if not rel.empty else None
    association_type = rel["Association"].iloc[0] if not rel.empty else None

    try:
        reg_complexity = fda_products_reduced[fda_products_reduced["norm_drug_name"] == drug].groupby("appl_no").size().mean()
    except Exception:
        reg_complexity = None

    for case_id, group in faers_cases.groupby("caseid"):
        events = group["pt"].dropna().tolist()
        severities = [map_severity(ev) for ev in events if isinstance(ev, str)]
        categories = []
        for ev in events:
            categories.extend(categorize_event(ev))

        has_severe = any(s in ["severe", "life-threatening"] for s in severities)
        has_cardiac = any("cardiac" in c for c in categories)
        has_gi = any("gi" in c for c in categories)
        max_sev = max(severities, key=lambda s: severity_rank.get(s, 0)) if severities else "unknown"

        first = group.iloc[0]
        agg_rows.append({
            "case_id": case_id,
            "drug": drug,
            "gene": gene,
            **expr_values,
            "age": first["age"],
            "sex": first["sex"],
            "evidence_level": evidence_level,
            "association_type": association_type,
            "approval_year": approval_year,
            "regulatory_complexity": reg_complexity,
            "event_count": len(events),
            "has_severe_event": has_severe,
            "has_cardiac_event": has_cardiac,
            "has_gi_event": has_gi,
            "max_severity": max_sev,
            "event_categories": list(set(categories))
        })

per_case_df = pd.DataFrame(agg_rows)
print(per_case_df.shape)
print(per_case_df.head(10))


In [ ]:
# Final dataset, this is where we save


# Output directory
output_dir = Path("datasets/processed/full_integration")
output_dir.mkdir(parents=True, exist_ok=True)

# Aggregation Function
def build_aggregated_dataset(pairs,
                             faers_reactions_reduced,
                             pharmgkb_relationships_reduced,
                             fda_products_reduced,
                             faers_drugs_reduced,
                             faers_demographics_reduced,
                             event_categories_map):
    agg_rows = []

    for drug, gene in tqdm(pairs, desc="Building aggregated dataset", leave=False):
        expr_values = get_expression(gene)
        approval_year = get_approval_year(drug)

        faers_cases = faers_drugs_reduced[faers_drugs_reduced["norm_drugname"] == drug]
        if faers_cases.empty:
            continue

        faers_cases = faers_cases.merge(faers_reactions_reduced, on=["primaryid", "caseid"], how="inner")
        faers_cases = faers_cases.merge(
            faers_demographics_reduced[["primaryid", "caseid", "age", "sex"]],
            on=["primaryid", "caseid"], how="left"
        )

        rel = pharmgkb_relationships_reduced[
            ((pharmgkb_relationships_reduced["Entity1_name"] == drug) &
             (pharmgkb_relationships_reduced["Entity2_name"] == gene)) |
            ((pharmgkb_relationships_reduced["Entity1_name"] == gene) &
             (pharmgkb_relationships_reduced["Entity2_name"] == drug))
        ]
        evidence_level = rel["Evidence"].iloc[0] if not rel.empty else None
        association_type = rel["Association"].iloc[0] if not rel.empty else None

        try:
            reg_complexity = fda_products_reduced[
                fda_products_reduced["norm_drug_name"] == drug
            ].groupby("appl_no").size().mean()
        except Exception:
            reg_complexity = None

        for case_id, group in faers_cases.groupby("caseid"):
            events = group["pt"].dropna().tolist()
            severities = [map_severity(ev) for ev in events if isinstance(ev, str)]
            categories = []
            for ev in events:
                categories.extend(categorize_event(ev))

            has_severe = any(s in ["severe", "life-threatening"] for s in severities)
            has_cardiac = any("cardiac" in c for c in categories)
            has_gi = any("gi" in c for c in categories)
            max_sev = max(severities, key=lambda s: severity_rank.get(s, 0)) if severities else "unknown"

            first = group.iloc[0]
            agg_rows.append({
                "case_id": case_id,
                "drug": drug,
                "gene": gene,
                **expr_values,
                "age": first["age"],
                "sex": first["sex"],
                "evidence_level": evidence_level,
                "association_type": association_type,
                "approval_year": approval_year,
                "regulatory_complexity": reg_complexity,
                "event_count": len(events),
                "has_severe_event": has_severe,
                "has_cardiac_event": has_cardiac,
                "has_gi_event": has_gi,
                "max_severity": max_sev,
                "event_categories": list(set(categories))
            })

    return pd.DataFrame(agg_rows)


# Parameters
batch_size = 200
pairs = list(drug_gene_pairs)

# Tracking
all_files = []

print(f"Total pairs: {len(pairs)}")
print(f"Processing in batches of {batch_size}...")

for i in range(0, len(pairs), batch_size):
    batch_pairs = pairs[i:i+batch_size]
    batch_id = f"batch_{i//batch_size+1:03d}"

    print(f"\nProcessing {batch_id} ({len(batch_pairs)} pairs)")

    batch_df = build_aggregated_dataset(
        batch_pairs,
        faers_reactions_reduced,
        pharmgkb_relationships_reduced,
        fda_products_reduced,
        faers_drugs_reduced,
        faers_demographics_reduced,
        event_categories_map
    )

    out_path = output_dir / f"faers_integration_{batch_id}.parquet"
    batch_df.to_parquet(out_path, index=False)
    all_files.append(out_path)

    print(f"Saved {len(batch_df):,} rows → {out_path}")

print("\nFinished all batches")
print(f"Total files saved: {len(all_files)}")
